# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [96]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [98]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,58.23,63,0,13.06,AU,1732727184
1,1,utrik,11.2278,169.8474,82.83,79,99,21.03,MH,1732727186
2,2,treinta y tres,-33.2333,-54.3833,74.07,88,100,4.52,UY,1732727189
3,3,port-aux-francais,-49.3500,70.2167,38.46,89,99,25.57,TF,1732727191
4,4,taco pozo,-25.6167,-63.2833,104.05,26,98,10.65,AR,1732727193


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [101]:
%%capture --no-display

# Configure the map plot
humidityPlot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True, 
    size= "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.7,
    tiles = "OSM",
    frame_height = 504,
    frame_width = 896,
    hover_cols = ["City", "Humidity"]
)

# Display the map
humidityPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [104]:
# Narrow down cities that fit criteria and drop any results with null values
idealCityDF = city_data_df.loc[
    (city_data_df["Max Temp"] > 69.8) & (city_data_df["Max Temp"] < 80.6)
    & (city_data_df["Wind Speed"] < 4.5)
    & (city_data_df["Cloudiness"] ==0)
]
# Drop any rows with null values
indealCityDF = indealCityDF.dropna()
# Display sample data
print(len(idealCityDF))
idealCityDF.head()

5


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
35,35,mangrol,21.1167,70.1167,77.65,52,0,1.16,IN,1732727263
163,163,la ermita,20.4139,-101.9936,74.97,21,0,1.16,MX,1732727541
516,516,settat,33.0007,-7.6204,75.13,29,0,4.36,MA,1732728317
524,524,sur,22.5667,59.5289,79.02,57,0,4.34,OM,1732728334
556,556,san julian,21.0167,-102.1667,70.36,19,0,2.62,MX,1732728403


### Step 3: Create a new DataFrame called `hotel_df`.

In [107]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealCityDF[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
35,mangrol,IN,21.1167,70.1167,52,
163,la ermita,MX,20.4139,-101.9936,21,
516,settat,MA,33.0007,-7.6204,29,
524,sur,OM,22.5667,59.5289,57,
556,san julian,MX,21.0167,-102.1667,19,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [115]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accomodation.hotel",
    "apiKey": geoapify_key, 
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude}, {latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!= "No hotel found"]
hotel_df

Starting hotel search


,City,Country,Lat,Lng,Humidity,Hotel Name


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [113]:
%%capture --no-display

# Configure the map plot
hotelPlot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = 1,
    color = "City Name",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 500,
    hover_cols = ["City Name", "Fahrenheit Temperature", "Hotel Name", "Hotel Address"]
)
hotelPlot
                  \

# Display the map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]